## TensorFlow2教程-Keras概述

Keras 是一个用于构建和训练深度学习模型的高阶 API。它可用于快速设计原型、高级研究和生产。

Keras的3个优点：
方便用户使用、模块化和可组合、易于扩展


##  1 导入tf.keras
TensorFlow2推荐使用tf.keras构建网络，常见的神经网络都包含在tf.keras.layer中(最新的tf.keras的版本可能和keras不同)

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
print(tf.__version__)
print(tf.keras.__version__)

2.8.0
2.8.0


## 2 构建简单模型
### 2.1 模型堆叠
最常见的模型类型是层的堆叠：tf.keras.Sequential 模型

In [2]:
model = tf.keras.Sequential()
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

### 2.2 网络配置

tf.keras.layers中主要的网络配置参数如下：

activation：设置层的激活函数。此参数可以是函数名称字符串，也可以是函数对象。默认情况下，系统不会应用任何激活函数。

kernel_initializer 和 bias_initializer：创建层权重（核和偏置）的初始化方案。此参数是一个名称或可调用的函数对象，默认为 "Glorot uniform" 初始化器。

kernel_regularizer 和 bias_regularizer：应用层权重（核和偏置）的正则化方案，例如 L1 或 L2 正则化。默认情况下，系统不会应用正则化函数。

In [3]:
layers.Dense(32, activation='sigmoid')
layers.Dense(32, activation=tf.sigmoid)
layers.Dense(32, kernel_initializer='orthogonal')
layers.Dense(32, kernel_initializer=tf.keras.initializers.glorot_normal)
layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l2(0.01))
layers.Dense(32, kernel_regularizer=tf.keras.regularizers.l1(0.01))

## 3 训练和评估
### 3.1 设置训练流程
构建好模型后，通过调用 compile 方法配置该模型的学习流程：

In [4]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.categorical_crossentropy,
    metrics=[tf.keras.metrics.categorical_accuracy]
)

### 3.2 输入Numpy数据
对于小型数据集，可以使用Numpy构建输入数据。

In [5]:
import numpy as np

train_x = np.random.random((1000, 72))
train_y = np.random.random((1000, 10))
val_x = np.random.random((200, 72))
val_y = np.random.random((200, 10))
model.fit(train_x, train_y, epochs=10, batch_size=100,
          validation_data=(val_x, val_y))

Epoch 1/10
10/10 [==============================] - 1s 20ms/step - loss: 11.6599 - categorical_accuracy: 0.0900 - val_loss: 11.4832 - val_categorical_accuracy: 0.1300
Epoch 2/10
10/10 [==============================] - 0s 4ms/step - loss: 11.7889 - categorical_accuracy: 0.1030 - val_loss: 11.8302 - val_categorical_accuracy: 0.1200
Epoch 3/10
10/10 [==============================] - 0s 5ms/step - loss: 12.4421 - categorical_accuracy: 0.1040 - val_loss: 13.0340 - val_categorical_accuracy: 0.1500
Epoch 4/10
10/10 [==============================] - 0s 5ms/step - loss: 14.4235 - categorical_accuracy: 0.0940 - val_loss: 16.1756 - val_categorical_accuracy: 0.1700
Epoch 5/10
10/10 [==============================] - 0s 4ms/step - loss: 18.5743 - categorical_accuracy: 0.0940 - val_loss: 21.3999 - val_categorical_accuracy: 0.1300
Epoch 6/10
10/10 [==============================] - 0s 4ms/step - loss: 24.5563 - categorical_accuracy: 0.0870 - val_loss: 28.0612 - val_categorical_accuracy: 0.1100
Epo

### 3.3 tf.data输入数据
对于大型数据集可以使用tf.data构建训练输入。

In [7]:
datasets = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
datasets = datasets.repeat()
val_datasets = tf.data.Dataset.from_tensor_slices((val_x, val_y)).batch(32)
val_datasets = val_datasets.repeat()

model.fit(datasets, epochs=10, steps_per_epoch=30, validation_data=val_datasets, validation_steps=3)

Epoch 1/10
30/30 [==============================] - 0s 7ms/step - loss: 96.0555 - categorical_accuracy: 0.0948 - val_loss: 125.4274 - val_categorical_accuracy: 0.1250
Epoch 2/10
30/30 [==============================] - 0s 2ms/step - loss: 159.0904 - categorical_accuracy: 0.0929 - val_loss: 203.0643 - val_categorical_accuracy: 0.1250
Epoch 3/10
30/30 [==============================] - 0s 3ms/step - loss: 242.9147 - categorical_accuracy: 0.0951 - val_loss: 296.2151 - val_categorical_accuracy: 0.1250
Epoch 4/10
30/30 [==============================] - 0s 3ms/step - loss: 338.8953 - categorical_accuracy: 0.0983 - val_loss: 397.7326 - val_categorical_accuracy: 0.1146
Epoch 5/10
30/30 [==============================] - 0s 3ms/step - loss: 440.1417 - categorical_accuracy: 0.0876 - val_loss: 502.6749 - val_categorical_accuracy: 0.0521
Epoch 6/10
30/30 [==============================] - 0s 3ms/step - loss: 544.6055 - categorical_accuracy: 0.0962 - val_loss: 621.9797 - val_categorical_accuracy: 

### 3.4 评估与预测
评估和预测函数：tf.keras.Model.evaluate和tf.keras.Model.predict方法，都可以可以使用NumPy和tf.data.Dataset构造的输入数据进行评估和预测

In [8]:
# 模型评估
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)

30/30 [==============================] - 0s 2ms/step - loss: 1074.1073 - categorical_accuracy: 0.1208


[1074.1072998046875, 0.12083332985639572]

In [9]:
# 模型预测
result = model.predict(test_x, batch_size=32)
print(result)

[[0.         0.16240874 0.08368254 ... 0.00671651 0.         0.        ]
 [0.         0.08923861 0.05535543 ... 0.00927343 0.         0.        ]
 [0.         0.15965922 0.06607018 ... 0.00557643 0.         0.        ]
 ...
 [0.         0.12314524 0.06737925 ... 0.00542385 0.         0.        ]
 [0.         0.15415026 0.08174875 ... 0.00448536 0.         0.        ]
 [0.         0.13542601 0.07227626 ... 0.00791746 0.         0.        ]]


## 4 构建复杂模型

### 4.1 函数式API
tf.keras.Sequential 模型是层的简单堆叠，无法表示任意模型。使用 Keras的函数式API可以构建复杂的模型拓扑，例如：

- 多输入模型，

- 多输出模型，

- 具有共享层的模型（同一层被调用多次），

- 具有非序列数据流的模型（例如，残差连接）。

**使用函数式 API 构建的模型具有以下特征：**

- 层实例可调用并返回张量。
- 输入张量和输出张量用于定义 tf.keras.Model 实例。
- 此模型的训练方式和 Sequential 模型一样。

In [ ]:
input_x = tf.keras.Input(shape=(72, ))
hidden1 = layers.Dense(32, activation='relu')(input_x)
hidden2 = layers.Dense(16, activation='relu')(hidden1)
